In [1]:
import numpy as np
from tensorflow import keras

In [2]:
def generate_time_series(batch_size, n_steps):
    freq1 ,freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offset2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)

train_X = series[:7500, :n_steps]
train_y = series[:7500, -1]
valid_X = series[7500:, :n_steps]
valid_y = series[7500:, -1]

In [4]:
valid_pred_naive_y = valid_X[:, -1]
np.mean(keras.losses.mean_squared_error(valid_y, valid_pred_naive_y))

0.021025112

In [5]:
early_stop_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [6]:
model_linear = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model_linear.compile(loss='mean_squared_error', optimizer='Adam')
model_linear.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 0s 906us/step - loss: 0.0754 - val_loss: 0.0275
Epoch 2/50
235/235 [==============================] - 0s 521us/step - loss: 0.0185 - val_loss: 0.0138
Epoch 3/50
235/235 [==============================] - 0s 538us/step - loss: 0.0111 - val_loss: 0.0101
Epoch 4/50
235/235 [==============================] - 0s 526us/step - loss: 0.0085 - val_loss: 0.0080
Epoch 5/50
235/235 [==============================] - 0s 530us/step - loss: 0.0071 - val_loss: 0.0069
Epoch 6/50
235/235 [==============================] - 0s 527us/step - loss: 0.0063 - val_loss: 0.0061
Epoch 7/50
235/235 [==============================] - 0s 530us/step - loss: 0.0056 - val_loss: 0.0055
Epoch 8/50
235/235 [==============================] - 0s 559us/step - loss: 0.0051 - val_loss: 0.0051
Epoch 9/50
235/235 [==============================] - 0s 523us/step - loss: 0.0047 - val_loss: 0.0048
Epoch 10/50
235/235 [==============================] - 0s 529us/step - loss: 0.004

In [7]:
model_baseline = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(1)
])

model_baseline.compile(loss='mean_squared_error', optimizer='Adam')
model_baseline.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 0s 952us/step - loss: 0.0453 - val_loss: 0.0144
Epoch 2/50
235/235 [==============================] - 0s 703us/step - loss: 0.0103 - val_loss: 0.0090
Epoch 3/50
235/235 [==============================] - 0s 732us/step - loss: 0.0071 - val_loss: 0.0069
Epoch 4/50
235/235 [==============================] - 0s 728us/step - loss: 0.0057 - val_loss: 0.0056
Epoch 5/50
235/235 [==============================] - 0s 718us/step - loss: 0.0048 - val_loss: 0.0049
Epoch 6/50
235/235 [==============================] - 0s 708us/step - loss: 0.0044 - val_loss: 0.0044
Epoch 7/50
235/235 [==============================] - 0s 716us/step - loss: 0.0040 - val_loss: 0.0042
Epoch 8/50
235/235 [==============================] - 0s 701us/step - loss: 0.0037 - val_loss: 0.0039
Epoch 9/50
235/235 [==============================] - 0s 723us/step - loss: 0.0036 - val_loss: 0.0040
Epoch 10/50
235/235 [==============================] - 0s 723us/step - loss: 0.003

In [8]:
model_simpleRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model_simpleRNN.compile(loss='mean_squared_error', optimizer='Adam')
model_simpleRNN.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 2s 5ms/step - loss: 0.1928 - val_loss: 0.1713
Epoch 2/50
235/235 [==============================] - 1s 4ms/step - loss: 0.1582 - val_loss: 0.1416
Epoch 3/50
235/235 [==============================] - 1s 4ms/step - loss: 0.1312 - val_loss: 0.1163
Epoch 4/50
235/235 [==============================] - 1s 4ms/step - loss: 0.1056 - val_loss: 0.0915
Epoch 5/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0825 - val_loss: 0.0706
Epoch 6/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0627 - val_loss: 0.0527
Epoch 7/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0461 - val_loss: 0.0381
Epoch 8/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0329 - val_loss: 0.0270
Epoch 9/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0234 - val_loss: 0.0196
Epoch 10/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0175 - val_loss: 0.0156

In [10]:
model_deepRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model_deepRNN.compile(loss='mean_squared_error', optimizer='Adam')
model_deepRNN.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 4s 13ms/step - loss: 0.0085 - val_loss: 0.0034
Epoch 2/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0031 - val_loss: 0.0030
Epoch 3/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0031 - val_loss: 0.0036
Epoch 4/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0029 - val_loss: 0.0030
Epoch 5/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0029 - val_loss: 0.0027
Epoch 6/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0029 - val_loss: 0.0028
Epoch 7/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0028 - val_loss: 0.0028
Epoch 8/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0029 - val_loss: 0.0028
Epoch 9/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0027 - val_loss: 0.0028
Epoch 10/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0028 - val_lo

In [11]:
model_singleLSTM = keras.models.Sequential([
    keras.layers.LSTM(1, input_shape=[None,1]),
])

model_singleLSTM.compile(loss='mean_squared_error', optimizer='Adam')
model_singleLSTM.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 4s 13ms/step - loss: 0.1378 - val_loss: 0.1313
Epoch 2/50
235/235 [==============================] - 3s 11ms/step - loss: 0.1240 - val_loss: 0.1160
Epoch 3/50
235/235 [==============================] - 2s 9ms/step - loss: 0.1088 - val_loss: 0.1030
Epoch 4/50
235/235 [==============================] - 2s 9ms/step - loss: 0.0968 - val_loss: 0.0923
Epoch 5/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0857 - val_loss: 0.0803
Epoch 6/50
235/235 [==============================] - 2s 11ms/step - loss: 0.0720 - val_loss: 0.0639
Epoch 7/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0552 - val_loss: 0.0469
Epoch 8/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0408 - val_loss: 0.0350
Epoch 9/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0312 - val_loss: 0.0276
Epoch 10/50
235/235 [==============================] - 2s 11ms/step - loss: 0.0251 - val_loss

In [12]:
model_deepLSTM = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None,1]),
    keras.layers.LSTM(20),
    keras.layers.Dense(1)
])

model_deepLSTM.compile(loss='mean_squared_error', optimizer='Adam')
model_deepLSTM.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 7s 23ms/step - loss: 0.0449 - val_loss: 0.0223
Epoch 2/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0173 - val_loss: 0.0111
Epoch 3/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0066 - val_loss: 0.0040
Epoch 4/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0034 - val_loss: 0.0038
Epoch 5/50
235/235 [==============================] - 5s 20ms/step - loss: 0.0030 - val_loss: 0.0028
Epoch 6/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0029 - val_loss: 0.0028
Epoch 7/50
235/235 [==============================] - 5s 20ms/step - loss: 0.0028 - val_loss: 0.0031
Epoch 8/50
235/235 [==============================] - 5s 20ms/step - loss: 0.0029 - val_loss: 0.0030
Epoch 9/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0029 - val_loss: 0.0031
Epoch 10/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0028 - val_lo